<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/longstaff_schwartz_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

def longstaff_schwartz(S0, K, sigma, T, r, Np, NT):
    """
    Longstaff-Schwartz algorithm implemented in PyTorch.

    Args:
        S0: Initial asset price.
        K: Strike price.
        sigma: Volatility.
        T: Time to maturity.
        r: Risk-free rate.
        Np: Number of simulated paths.
        NT: Number of time steps.

    Returns:
        V: Option value.
    """
    dt = T / torch.tensor(NT, dtype=torch.float32)  # Ensure dt is a tensor
    sqrt_dt = torch.sqrt(dt)

    # Simulate paths
    Z = torch.randn(Np, NT)
    Sp = torch.zeros(Np, NT, dtype=torch.float32)
    Sp[:, 0] = S0

    for t in range(1, NT):
        previous_step = Sp[:, t - 1].clone()  # Avoid modifying previous step
        Sp[:, t] = previous_step * torch.exp((r - 0.5 * sigma**2) * dt + sigma * sqrt_dt * Z[:, t])

    # Initialize cash flows
    cash_flow = torch.maximum(K - Sp[:, -1], torch.tensor(0.0, dtype=torch.float32))
    discount_factor = torch.exp(-r * dt)

    # Backward induction
    cash_flow = cash_flow.clone()  # Ensure no inplace modification
    for t in range(NT - 2, 0, -1):
        in_the_money = Sp[:, t] < K
        itm_indices = torch.where(in_the_money)[0]

        if len(itm_indices) > 0:
            X = Sp[itm_indices, t]
            Y = cash_flow[itm_indices] * discount_factor.clone()

            # Regression to approximate continuation value
            A = torch.stack([torch.ones_like(X), X, X**2], dim=1)
            coeffs = torch.linalg.lstsq(A, Y).solution

            continuation_value = coeffs[0] + coeffs[1] * X + coeffs[2] * X**2

            exercise_value = K - X

            exercise = exercise_value > continuation_value
            exercise_indices = itm_indices[exercise]

            cash_flow = cash_flow.clone()  # Avoid inplace modification
            cash_flow[exercise_indices] = exercise_value[exercise]

        cash_flow = cash_flow * discount_factor.clone()  # Ensure no inplace modification

    # Final option value
    V = cash_flow.mean() * torch.exp(-r * dt)
    return V

def calculate_sensitivities(S0, K, sigma, T, r, Np, NT):
    """
    Calculate sensitivities (Delta, Vega, Rho, Theta) using automatic differentiation.

    Args:
        S0: Initial asset price.
        K: Strike price.
        sigma: Volatility.
        T: Time to maturity.
        r: Risk-free rate.
        Np: Number of simulated paths.
        NT: Number of time steps.

    Returns:
        sensitivities: Dictionary containing Delta, Vega, Rho, Theta.
    """
    S0_t = torch.tensor(S0, requires_grad=True, dtype=torch.float32)
    sigma_t = torch.tensor(sigma, requires_grad=True, dtype=torch.float32)
    r_t = torch.tensor(r, requires_grad=True, dtype=torch.float32)
    T_t = torch.tensor(T, requires_grad=True, dtype=torch.float32)

    # Enable anomaly detection
    with torch.autograd.set_detect_anomaly(True):
        # Compute option value
        V = longstaff_schwartz(S0_t, K, sigma_t, T_t, r_t, Np, NT)

        # Compute gradients
        V.backward()

    delta = S0_t.grad.item()
    vega = sigma_t.grad.item()
    rho = r_t.grad.item()
    theta = T_t.grad.item()

    return {
        "Delta": delta,
        "Vega": vega,
        "Rho": rho,
        "Theta": theta
    }

# Parameters for testing
S0 = 100.0
K = 95.0
sigma = 0.25
T = 180 / 365
r = 0.05
Np = 5000
NT = 1000

# Test the algorithm
option_value = longstaff_schwartz(S0, K, sigma, T, r, Np, NT)
sensitivities = calculate_sensitivities(S0, K, sigma, T, r, Np, NT)

print(f"Option Value: {option_value}")
print(f"Sensitivities: {sensitivities}")


Estimated Option Value: 1.1780


In [ ]:
!git clone https://github.com/luphord/longstaff_schwartz.git

Cloning into 'longstaff_schwartz'...
remote: Enumerating objects: 1046, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 1046 (delta 128), reused 221 (delta 114), pack-reused 791 (from 1)
Receiving objects: 100% (1046/1046), 12.07 MiB | 14.58 MiB/s, done.
Resolving deltas: 100% (642/642), done.


In [ ]:
!ls -ltr

total 8
drwxr-xr-x 1 root root 4096 Jan 16 14:29 sample_data
drwxr-xr-x 9 root root 4096 Jan 21 16:54 longstaff_schwartz


In [ ]:
!pip install ./longstaff_schwartz

In [ ]:
%cd longstaff_schwartz

/content/longstaff_schwartz


In [ ]:
from longstaff_schwartz.algorithm import longstaff_schwartz
from longstaff_schwartz.stochastic_process import GeometricBrownianMotion
import numpy as np


T = 180/365
#Np = 5000, NT = 1000.
# Model parameters
t = np.linspace(0, T, 5000)  # timegrid for simulation
r = 0.05  # riskless rate
sigma = 0.25  # annual volatility of underlying
n = 5000  # number of simulated paths
S0 = 100.0
K  = 95.0

# Simulate the underlying
gbm = GeometricBrownianMotion(mu=r, sigma=sigma)
rnd = np.random.RandomState(1234)
x = gbm.simulate(t, n, rnd)  # x.shape == (t.size, n)

x *= S0

# Payoff (exercise) function
strike = 95.0

def put_payoff(spot):
    return np.maximum(strike - spot, 0.0)

# Discount factor function
def constant_rate_df(t_from, t_to):
    return np.exp(-r * (t_to - t_from))

# Approximation of continuation value
#def fit_quadratic(x, y):
#    return np.polynomial.Polynomial.fit(x, y, 2, rcond=None)

# Approximation of continuation value
def fit_quadratic(x, y):
    if len(x) == 0 or len(y) == 0:
        return lambda x: np.zeros_like(x, dtype=float)
    try:
        # Attempt to fit using the original method
        return np.polynomial.Polynomial.fit(x, y, 2, rcond=None)
    except np.linalg.LinAlgError:
        # If SVD fails, use a more robust method like 'lstsq' directly
        A = np.vstack([x**0, x**1, x**2]).T  # Construct the design matrix
        coeffs, _, _, _ = np.linalg.lstsq(A, y, rcond=None)
        return np.polynomial.Polynomial(coeffs)  # Return a Polynomial object

# Selection of paths to consider for exercise
# (and continuation value approxmation)
def itm(payoff, spot):
    return payoff > 0

# Run valuation of American put option
npv_american = longstaff_schwartz(x, t, constant_rate_df,
                                  fit_quadratic, put_payoff, itm)

# European put option for comparison
npv_european = constant_rate_df(t[0], t[-1]) * put_payoff(x[-1]).mean()

# Check results
#assert np.round(npv_american, 4) == 0.0734
#assert np.round(npv_european, 4) == 0.0626
#assert npv_american > npv_european

print(f"npv_american: {npv_american:.4f}")
print(f"npv_european: {npv_european:.4f}")

npv_american: 3.7927
npv_european: 3.6678


/usr/local/lib/python3.11/dist-packages/numpy/polynomial/polynomial.py:1362: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)
/usr/local/lib/python3.11/dist-packages/numpy/polynomial/polyutils.py:303: RuntimeWarning: divide by zero encountered in scalar divide
  off = (old[1]*new[0] - old[0]*new[1])/oldlen
/usr/local/lib/python3.11/dist-packages/numpy/polynomial/polyutils.py:304: RuntimeWarning: divide by zero encountered in scalar divide
  scl = newlen/oldlen
/usr/local/lib/python3.11/dist-packages/numpy/polynomial/polyutils.py:372: RuntimeWarning: invalid value encountered in add
  return off + scl*x
